In [ ]:
import os
import sys
import subprocess

# Library installation (force Numpy < 2.0)
# Use subprocesses to ensure that it finishes before continuing
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "git+https://github.com/THU-MIG/yolov10.git"])
subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "ray"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy<2.0", "pandas", "--force-reinstall"])

# Download the weights
if not os.path.exists('yolov10n.pt'):
    subprocess.check_call(["wget", "-q", "https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt"])

In [ ]:
%%writefile train.py
import os
import glob
import random
import yaml
import torch
import numpy
from ultralytics import YOLO
from ultralytics import settings
from ultralytics.utils.loss import v8DetectionLoss



# Configuration
EXPERIMENT = '100pct_nms'
EPOCHS = 100
BATCH_SIZE = 16

if numpy.__version__ >= "2.0.0":
    raise ImportError(" ERRORE: Numpy è ancora > 2.0!")

# WANDB setup
os.environ['WANDB_MODE'] = 'disabled'
settings.update({"raytune": False})

# Patch for PyTorch
_original_load = torch.load
def safe_load(*args, **kwargs):
    if 'weights_only' not in kwargs:
        kwargs['weights_only'] = False
    return _original_load(*args, **kwargs)
torch.load = safe_load

# Dataset management
dataset_path = ''
potential_paths = [
    './datasets/VisDrone',
    '/kaggle/input/visdrone/VisDrone',
    '/kaggle/working/datasets/VisDrone'
]

# If there's no dataset, download YOLO dummy
if not any(os.path.exists(p) for p in potential_paths):
    try:
        model_dummy = YOLO('yolov10n.pt')
        model_dummy.train(data='VisDrone.yaml', epochs=1, imgsz=32, batch=1)
    except:
        print("   (Download trigger completato o interrotto)")

for path in potential_paths:
    if os.path.exists(path):
        dataset_path = path
        break

if not dataset_path:
    dataset_path = '/kaggle/working/datasets/VisDrone'

# Data preparation
train_images = glob.glob(os.path.join(dataset_path, 'VisDrone2019-DET-train/images/*.jpg'))
val_images = glob.glob(os.path.join(dataset_path, 'VisDrone2019-DET-val/images/*.jpg'))

if not train_images:
    raise FileNotFoundError(" ERRORE: Nessuna immagine trovata!")

random.seed(42)
random.shuffle(train_images)

selected_images = train_images
print(f" Training su {EXPERIMENT}: {len(selected_images)} immagini (FULL DATASET).")

# Writing .txt files
cwd = os.getcwd()
visdrone_classes = {
    0: 'pedestrian', 1: 'people', 2: 'bicycle', 3: 'car', 4: 'van',
    5: 'truck', 6: 'tricycle', 7: 'awning-tricycle', 8: 'bus', 9: 'motor'
}

txt_filename = os.path.join(cwd, f'train_{EXPERIMENT}.txt')
with open(txt_filename, 'w') as f:
    f.write('\n'.join(selected_images))

val_txt = os.path.join(cwd, 'val_official.txt')
with open(val_txt, 'w') as f:
    f.write('\n'.join(val_images))

# YAML writing
yaml_content = {
    'path': cwd,
    'train': txt_filename,
    'val': val_txt,
    'names': visdrone_classes
}
yaml_filename = os.path.join(cwd, f'VisDrone_{EXPERIMENT}.yaml')
with open(yaml_filename, 'w') as f:
    yaml.dump(yaml_content, f)

# Forces the use of the correct Loss v8 for both training and validation
def force_v8_loss(trainer):
    
    # Patch Base Model (Training)
    if hasattr(trainer, 'model'):
        try:
            trainer.model.criterion = v8DetectionLoss(trainer.model)
            if hasattr(trainer, 'loss'):
                trainer.loss = trainer.model.criterion
        except Exception as e:
            print(f" Impossibile patchare Modello Base: {e}")

    # Patch Modello EMA (Validazione) 
    if hasattr(trainer, 'ema') and trainer.ema:
        try:
            ema_model = getattr(trainer.ema, 'ema', None) or getattr(trainer.ema, 'model', None)
            if ema_model:
                ema_model.criterion = v8DetectionLoss(ema_model)
            else:
                print(" Modello EMA non trovato.")
        except Exception as e:
            print(f" Errore nel patchare l'EMA: {e}")
            
# NMS Model Construction
# Download the original configuration
if not os.path.exists('yolov10n.yaml'):
    os.system("wget -q https://raw.githubusercontent.com/THU-MIG/yolov10/main/ultralytics/cfg/models/v10/yolov10n.yaml -O yolov10n.yaml")

# Modify the configuration to use the standard head (Detect) instead of v10Detect
if os.path.exists('yolov10n.yaml'):
    with open('yolov10n.yaml', 'r') as f:
        config_str = f.read()

    # This replacement forces the use of the classic v8 head (with NMS).
    nms_config_str = config_str.replace('v10Detect', 'Detect')

    with open('yolov10n_nms.yaml', 'w') as f:
        f.write(nms_config_str)
else:
    raise FileNotFoundError("ERRORE: Impossibile creare yolov10n_nms.yaml")

# Instantiate the model with the new architecture
model_nms = YOLO('yolov10n_nms.yaml')

# Callback registration
model_nms.add_callback("on_train_start", force_v8_loss)

# Download original weight
if not os.path.exists('yolov10n.pt'):
     os.system("wget -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt")

# Load the weight
ckpt = torch.load('yolov10n.pt', map_location='cpu')
state_dict = ckpt['model'].float().state_dict() if 'model' in ckpt else ckpt

missing, unexpected = model_nms.model.load_state_dict(state_dict, strict=False)



# Training
model_nms.train(
    data=yaml_filename,      
    epochs=EPOCHS,           
    imgsz=640,
    batch=BATCH_SIZE,        
    device=0,
    optimizer='AdamW',
    lr0=0.001,               # Learning rate standard
    freeze=0,            
    patience=10,
    project='YOLOv10_VisDrone',
    name=f'run_{EXPERIMENT}',
    exist_ok=True,
    verbose=True,
    val=True,                
    plots=True
)

In [ ]:
!python train.py
